In [4]:
import keras

In [5]:
from keras.datasets import mnist

In [6]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from keras import backend as k
from keras import optimizers,losses
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta

In [7]:
# splitting data into traain and test
(x_train,y_train),(x_test,y_test)= mnist.load_data()

In [8]:
x_train = x_train.reshape(x_train.shape[0],28,28,1)
x_test = x_test.reshape(x_test.shape[0],28,28,1)

In [9]:
input_shape =(28,28,1)

In [10]:
#converting class to binary
y_train=keras.utils.to_categorical(y_train,10)
y_test = keras.utils.to_categorical(y_test,10)

In [11]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [12]:
x_train/=255
x_test/=255

In [13]:
batch_size = 128
no_classes =10
epochs = 10
model = Sequential()

In [14]:
model.add(Conv2D(filters = 32,kernel_size=(5, 5), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))

In [15]:
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(no_classes,activation='softmax'))

In [16]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

In [17]:
hist = model.fit(x_train,y_train,batch_size=batch_size,
                 epochs=epochs,verbose=1,validation_data=(x_test,y_test))


Epoch 1/10
469/469 [==============================] - 32s 62ms/step - loss: 2.3027 - accuracy: 0.1122 - val_loss: 2.2805 - val_accuracy: 0.1654
Epoch 2/10
469/469 [==============================] - 27s 58ms/step - loss: 2.2727 - accuracy: 0.1531 - val_loss: 2.2472 - val_accuracy: 0.2794
Epoch 3/10
469/469 [==============================] - 27s 57ms/step - loss: 2.2421 - accuracy: 0.2003 - val_loss: 2.2110 - val_accuracy: 0.4000
Epoch 4/10
469/469 [==============================] - 27s 57ms/step - loss: 2.2099 - accuracy: 0.2411 - val_loss: 2.1681 - val_accuracy: 0.5037
Epoch 5/10
469/469 [==============================] - 27s 57ms/step - loss: 2.1695 - accuracy: 0.2838 - val_loss: 2.1161 - val_accuracy: 0.5758
Epoch 6/10
469/469 [==============================] - 27s 57ms/step - loss: 2.1246 - accuracy: 0.3223 - val_loss: 2.0573 - val_accuracy: 0.6288
Epoch 7/10
469/469 [==============================] - 27s 58ms/step - loss: 2.0759 - accuracy: 0.3510 - val_loss: 1.9940 - val_accuracy:

In [18]:
score = model.evaluate(x_test,y_test,verbose=0)
print("loss",score[0])
print("accuracy",score[1])
model.save('mnist.h5')

loss 1.7854294776916504
accuracy 0.7185999751091003


In [19]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

# Load the pre-trained model
model = load_model('mnist.h5')

def predict_digit(img):
    # Resize image to 28x28 pixels
    img = img.resize((28, 28))
    # Convert RGB to grayscale
    img = img.convert('L')
    img = np.array(img)
    # Reshaping to support our model input and normalizing
    img = img.reshape(1, 28, 28, 1)
    img = img / 255.0
    # Predict the class
    res = model.predict(img)[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.x = self.y = 0
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg="white", cursor="cross")
        self.label = tk.Label(self, text="Draw a digit", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text="Recognize", command=self.classify_handwriting)
        self.button_clear = tk.Button(self, text="Clear", command=self.clear_all)
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W)
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r = 8
        self.canvas.create_oval(self.x - r, self.y - r, self.x + r, self.y + r, fill='black')

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()  # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND)  # get the coordinate of the canvas
        im = ImageGrab.grab(rect)
        digit, confidence = predict_digit(im)
        self.label.configure(text=f"Predicted digit: {digit}\nConfidence: {confidence*100:.2f}%")

    def clear_all(self):
        self.canvas.delete("all")

if __name__ == "__main__":
    app = App()
    mainloop()


1/1 [==============================] - 0s 78ms/step
